In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(['seaborn-ticks', 'seaborn-talk'])

import numpy as np
import pandas as pd
import xarray as xr

# Data Processing

Here, we take some of the archived data from Fernando's IGSM-CAM-GEOS-Chem ensemble and re-process it a bit to create a simpler dataset for playing with statistical meteorology-PM2.5 relationships. We similarly will extract some of Lu Shen's post-processed observational data, which he used in his 2017 paper.

Our major goal here is to subset the data for a small box around Boston.

In [5]:
fgm_ensemble = xr.open_dataset("../data/fgm.all_cases.usa_subset.nc")
fgm_ensemble

<xarray.Dataset>
Dimensions:  (dec: 3, ic: 5, lat: 15, lon: 24, pol: 3, time: 359)
Coordinates:
  * pol      (pol) object 'REF' 'P37' 'P45'
    cs       int64 ...
  * ic       (ic) int64 1 2 3 4 5
  * dec      (dec) object '1980-2010' '2035-2065' '2085-2115'
  * lat      (lat) float64 23.68 25.58 27.47 29.37 31.26 33.16 35.05 36.95 ...
  * lon      (lon) float64 -125.0 -122.5 -120.0 -117.5 -115.0 -112.5 -110.0 ...
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 1981-03-01 ...
    lev      float64 ...
Data variables:
    PM25     (pol, ic, dec, time, lat, lon) float64 ...
    TEMP     (pol, ic, dec, time, lat, lon) float64 ...
    PRECIP   (pol, ic, dec, time, lat, lon) float64 ...
    RH       (pol, ic, dec, time, lat, lon) float64 ...
    U        (pol, ic, dec, time, lat, lon) float64 ...
    V        (pol, ic, dec, time, lat, lon) float64 ...
Attributes:
    History:  Generated by 09_lu_shen_reproduction.ipynb (Apr 19, 2017 13:49)\n

In [4]:
fgm_ensemble.time

<xarray.DataArray 'time' (time: 1795)>
array(['1981-01-01T00:00:00.000000000', '1981-02-01T00:00:00.000000000',
       '1981-03-01T00:00:00.000000000', ..., '2126-09-01T00:00:00.000000000',
       '2126-10-01T00:00:00.000000000', '2126-11-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    lev      float64 992.6
    ic       (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 1981-03-01 ...
Attributes:
    long_name:  time
    bounds:     time_bnds

# Basic Example

For a simple example of how to use this toolkit, let's look at the following simple Principal Component Regression model, adapted from the example in [stat_pm25/models/simple.py]():

In [7]:
from stat_pm25.sklearn import *

class PCRModel(DatasetModel):
    """ Performs principal component regression model on gridded data.

    This class uses the `DatasetModel` framework to implement a principal
    component regression - a linear regression on a set of features which has
    been pre-processed using principal component analysis. It only requires a
    single additional argument on top of `DatasetModel`, the number of
    components to retain.

    This example illustrates how you can very easily put together complex
    analyses and deploy them onto your gridded model output, all using very
    simple building blocks.

    """

    def __init__(self, *args, n_components=3, **kwargs):
        # If you need to add arguments, add them as named keyword arguments in
        # the appropriate place (as shown above), and set them first in the
        # method.
        self.n_components = n_components

        # Modify any pre-set parameters, or hard-code them otherwise. For
        # instance, if you want to pre-process your data, this would be the
        # place to specify how to do so. Doing so here has the advantage that
        # you will be able to immediately apply your `predict()` method
        # to new data without pre-processing it - all that logic will be
        # saved

        # Zero out dilat and dilon, since we don't need to search around
        # neighboring grid cells
        self.dilat = self.dilon = 0

        # Set a pre-processor pipeline
        self.preprocessor = None
        
        # Call the parent superconstructor
        super().__init__(*args, **kwargs)

    def cell_kernel(self, gcf):
        """ Fit a model at a single grid cell.

        """

        # First, get the predictand data at the grid cell we care about. We
        # don't necessarily have to be super pedantic about this; we can just
        # use normal xarray selection methods if we want, although comments
        # below is how we could accomplish this using our specialized
        # Transformer classes
        # local_selector = DatasetSelector(
        #     sel='isel', lon=gcf.ilon, lat=gcf.ilat
        # )
        # y = local_selector.fit_transform(self.data[self.predictand])
        y = self.data[self.predictand].isel(lat=gcf.ilat, lon=gcf.ilon)

        # Prepare features timeseries. We want to fully include all the steps
        # to extract our features from the original, full dataset in here
        # so that our logic for re-applying the pipeline for prediction
        # later on will work similarly
        _model = Pipeline([
            ('subset_latlon', DatasetSelector(
                sel='isel', lon=gcf.ilon, lat=gcf.ilat)
            ),
            ('predictors', FieldExtractor(self.predictors)),
            ('normalize', Normalizer()),
            ('dataset_to_array', DatasetAdapter(drop=['lat', 'lon'])),
            ('pca', PCA(n_components=self.n_components)),
            ('linear', LinearRegression()),
        ])

        # Fit the model/pipeline
        _model.fit(self.data, y)
        # Calculate some sort of score for archival
        _score = _model.score(self.data, y)
        # Encapsulate the result within a GridCellResukt
        gcr = GridCellResult(_model, self.predictand, self.predictors, _score)

        return gcr

The only difference between this and the original, saved version is that I've excluded the pre-processing step - we can do that on our own.